In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import load_metric

f:\dev\Proyecto_MeIA\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel('datasets/MeIA_Rest_Mex_Sentiment_Analysis_2023_Train.xlsx', index_col=0)

In [3]:
print("Número de instacias: ",len(data))
print("Distribución de instancias por clase:")
print(data["Class"].value_counts())

Número de instacias:  17500
Distribución de instancias por clase:
Class
5    5250
4    4375
3    3500
2    2625
1    1750
Name: count, dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
a,b = train_test_split(data, test_size=0.30, stratify = data['Class'])
print("Longitud del conjunto de entrenamiento (70%): ",len(a))
print("Longitud del conjunto de pruebas (30%): ",len(b))
print(len(a)+len(b))

Longitud del conjunto de entrenamiento (70%):  12250
Longitud del conjunto de pruebas (30%):  5250
17500


In [5]:
print("Distribución de instancias del conjunto de entrenamiento por clase:")
print(a["Class"].value_counts())

Distribución de instancias del conjunto de entrenamiento por clase:
Class
5    3675
4    3063
3    2450
2    1837
1    1225
Name: count, dtype: int64


In [6]:
ds_train = Dataset.from_pandas(a)
ds_test = Dataset.from_pandas(b)
ds_dict = {'train' : ds_train, 'test':ds_test}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['Class', 'Review'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['Class', 'Review'],
        num_rows: 5250
    })
})

In [7]:
dataset = dataset.rename_column("Class", "labels")
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'Review'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'Review'],
        num_rows: 5250
    })
})

In [8]:
def polarity_init(example):
  example["labels"] = example["labels"] - 1
  return example

dataset = dataset.map(polarity_init)

dataset.set_format("pandas")
df = dataset["train"][:]
print(df["labels"].value_counts())
dataset.reset_format()

labels
4    3675
3    3063
2    2450
1    1837
0    1225
Name: count, dtype: int64


In [9]:
model_checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.vocab_size

119547

In [10]:
def tokenize_reviews(examples):
    return tokenizer(examples["Review"], truncation=True)

columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5250
    })
})

In [11]:
encoded_dataset["train"][0]

{'labels': 0,
 'input_ids': [101,
  107709,
  25694,
  10403,
  45075,
  10333,
  169,
  15046,
  19157,
  106,
  106,
  117,
  10911,
  96684,
  11620,
  10104,
  10109,
  107562,
  10104,
  39902,
  12186,
  26715,
  119,
  107709,
  13987,
  10638,
  89729,
  169,
  10109,
  107397,
  66589,
  13672,
  193,
  10125,
  24693,
  119,
  10183,
  10406,
  12921,
  10125,
  10121,
  16217,
  10109,
  22469,
  23641,
  19522,
  19975,
  10104,
  10109,
  107562,
  10104,
  39902,
  10553,
  10125,
  10109,
  44560,
  119,
  13069,
  10140,
  11107,
  12097,
  169,
  10109,
  40589,
  11266,
  10549,
  13956,
  10121,
  10141,
  12229,
  25969,
  10263,
  44024,
  37854,
  193,
  10183,
  10198,
  36101,
  10192,
  86286,
  56963,
  19618,
  24256,
  119,
  19803,
  11876,
  30785,
  46779,
  10110,
  10182,
  10150,
  11278,
  10121,
  12556,
  10157,
  169,
  17226,
  119,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [12]:
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.wei

In [13]:
metric = load_metric("f1")
metric

C:\Users\Home\AppData\Local\Temp\ipykernel_5204\846349347.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


Metric(name: "f1", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel targets. If set to `None`, the sco

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [19]:
model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
train_dataset = encoded_dataset["train"]
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps
)

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [21]:
trainer.train()

f:\dev\Proyecto_MeIA\env\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 12%|█▏        | 191/1532 [30:13<2:29:28,  6.69s/it]

{'loss': 1.414, 'learning_rate': 1.7506527415143603e-05, 'epoch': 0.25}


 25%|██▍       | 382/1532 [54:20<2:28:47,  7.76s/it]

{'loss': 1.2408, 'learning_rate': 1.5013054830287209e-05, 'epoch': 0.5}


 37%|███▋      | 573/1532 [1:19:01<2:30:14,  9.40s/it]

...

                                                     
100%|██████████| 1532/1532 [3:45:06<00:00,  7.33s/it]

{'eval_loss': 1.0352400541305542, 'eval_f1': 0.5414521352077455, 'eval_runtime': 892.1286, 'eval_samples_per_second': 5.885, 'eval_steps_per_second': 0.369, 'epoch': 2.0}


100%|██████████| 1532/1532 [3:45:10<00:00,  8.82s/it]

{'train_runtime': 13510.6668, 'train_samples_per_second': 1.813, 'train_steps_per_second': 0.113, 'train_loss': 1.1111767235997452, 'epoch': 2.0}


TrainOutput(global_step=1532, training_loss=1.1111767235997452, metrics={'train_runtime': 13510.6668, 'train_samples_per_second': 1.813, 'train_steps_per_second': 0.113, 'train_loss': 1.1111767235997452, 'epoch': 2.0})

In [22]:
eval_results = trainer.evaluate()
print(eval_results)

100%|██████████| 329/329 [19:38<00:00,  3.58s/it]

{'eval_loss': 1.0352400541305542, 'eval_f1': 0.5414521352077455, 'eval_runtime': 1180.265, 'eval_samples_per_second': 4.448, 'eval_steps_per_second': 0.279, 'epoch': 2.0}


In [24]:
data_test = pd.read_excel('datasets/MeIA_Rest_Mex_Sentiment_Analysis_2023_Test.xlsx', index_col=0)

In [25]:
trainer.save_model('models/')

In [26]:
from transformers import pipeline
pi = pipeline("text-classification", model="models/", max_length=512, truncation=True)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [27]:
text = "Bueno es un. Hotel que ya es necesario un mantenimiento y una modernización urgente porque con el nivel que esta presentando a hora no le alcanza ni par un 3 estrella sus ascensores son deplorable  y tiene un mal servicio de su personal"
res = pi(text)
print(res)

[{'label': 'LABEL_2', 'score': 0.43483832478523254}]


In [28]:
#Abrir archivo de texto en modo escritura para almacenar los resultados para el reto
with open('res/Resultados_MeIATeam_ModeloBase.txt', 'w') as f:
  for i in range(len(data_test)):
      print(i," - ",data_test.iloc[i]['Review'])
      res = pi(data_test.iloc[i]['Review'])
      label = res[0]['label']
      #Establecer la etiqueta correspondiente
      if label == 'LABEL_0':
        label = "1"
      elif label == 'LABEL_1':
        label = "2"
      elif label == 'LABEL_2':
        label = "3"
      elif label == 'LABEL_3':
        label = "4"
      elif label == 'LABEL_4':
        label = "5"
      #Escribir predicción del sentimiento en el archivo de texto con el formaro correspondiente
      f.write(str(i)+"\t"+label+"\n")
      #Imprimir en pantalla lo que se escribio en el archivo txt
      print(str(i)+"\t"+label+"\n")
      #Descomente si solo quiere probar este bloque de codigo para las primeras tres instancias de pruebas
      #if i ==2:
        #break

0  -  Definitivamente un lugar para NO volver .... lastimosamente me lleve una muy mala imagen del lugar por infinidad de cosas.
1. Los meseros que atendieron mi mesa no tenían idea de los platos que ofrecían en la carta y ni idea de atención al cliente...Más
0	1

1  -  Bueno es un. Hotel que ya es necesario un mantenimiento y una modernización urgente porque con el nivel que esta presentando a hora no le alcanza ni par un 3 estrella sus ascensores son deplorable  y tiene un mal servicio de su personal
1	3

2  -  Hoy fui a comer con mi novio a este lugar,  pedimos enchiladas, pero tres enchiladas estaban crudas, le dije a la mesera, lo mínimo que esperaba era que me cambiaran esas enchiladas por unas cosidas, lo único que recibí, fue mil disculpas, le comentaré a...Más
2	1

3  -  Estuve en la noche del 22 de sepbre.
Pésima atención.
Pésima comida.
SÚPER CARO. De verdad, ya los restaurantes como este, atracan, no prestan servicio.
Muy MALO. y pongo mi nombre. 
SERGIO OSORIO
3	1

4  -  P